[](https://imgur.com/7KH8RLQ)

# Preprocessing des donnees textuelle
Ici je vous présente toutes les étapes nécessaires pour traiter les données textuelles. 

**Rentrons dans le contexte du tutoriel:**

Nous avons une base de données nommée **spam.csv** qui regroupe des données sur des SMS qui sont catégorisés entre Spam et Ham. 
Le but est de traiter ces données.

# Charger les données
Avant de charger la base de données vous devez changer les paramètres de pour pandas pour voir les message en complet puis vous charger les données.

In [1]:
import pandas as pd
import numpy as np

#Pour afficher suffisament de textes dans les differents messages vous devez donner le width maximal pour les colonnes.
pd.set_option("display.max_colwidth", -1)
# Vous chargez les donnees mais les textes ne sont pas encode donc vous devez les encodes en "ISO-8859-1" et non "utf-8"
df = pd.read_csv("spam/spam.csv", encoding="ISO-8859-1")
df = df[["v1", "v2"]]
print("Le shape du dataframe: ", df.shape)
df.head(10)

Le shape du dataframe:  (5572, 2)


C:\Users\MOHAM\AppData\Local\Temp\ipykernel_14132\3908958675.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030


# Clean the dataset
Ici vous devez nettoyer l'ensemble des messages en transformant les messages et supprimant les caractères inutiles pour les modèles (Les ponctuations, mettre les phrases en minuscule, Tokenizer les caractères, retirer les mots d'arrêt, les liens, les hypers caractères et normaliser les données).

### Suppression des ponctuation
Pour supprimer les ponctuation nous utilisons la librairie **String**

In [2]:
import string

#Developpez une fonction qui parcoureras la base de donnees pour retirer les ponctuations 
def remove_punctuation(text):
    remove_ = "".join([i for i in text if i not in string.punctuation])
    return remove_

df["no_punctuation"] = df["v2"].apply(lambda x: remove_punctuation(x))
df.head(10)

,v1,v2,no_punctuation
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,As per your request Melle Melle Oru Minnaminunginte Nurungu Vettam has been set as your callertune for all Callers Press 9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.,WINNER As a valued network customer you have been selected to receivea å£900 prize reward To claim call 09061701461 Claim code KL341 Valid 12 hours only
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,Had your mobile 11 months or more U R entitled to Update to the latest colour mobiles with camera for Free Call The Mobile Update Co FREE on 08002986030


### Mettre en minuscule l'ensemble des messages

In [3]:
#Ce sript va parcourir l'ensembles de la base de donnees pour mettre tous les messages en minuscule
df["to_lowercase"] = df["no_punctuation"].apply(lambda x: x.lower())
df.head(10)

,v1,v2,no_punctuation,to_lowercase
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat,go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,nah i dont think he goes to usf he lives around here though
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv,freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent,even my brother is not like to speak with me they treat me like aids patent
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,As per your request Melle Melle Oru Minnaminunginte Nurungu Vettam has been set as your callertune for all Callers Press 9 to copy your friends Callertune,as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.,WINNER As a valued network customer you have been selected to receivea å£900 prize reward To claim call 09061701461 Claim code KL341 Valid 12 hours only,winner as a valued network customer you have been selected to receivea å£900 prize reward to claim call 09061701461 claim code kl341 valid 12 hours only
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,Had your mobile 11 months or more U R entitled to Update to the latest colour mobiles with camera for Free Call The Mobile Update Co FREE on 08002986030,had your mobile 11 months or more u r entitled to update to the latest colour mobiles with camera for free call the mobile update co free on 08002986030


### La tokenization
Dans cette étape vous allez tokenizer chaque mot de chacune des phrases, ce qui permettra à chaque mot d'avoir un ID pour les identifier et ainsi aider le modèle a reconnaitre les mots

In [4]:
#Tokensization des differents mots de chaque messages
import re

def tokenisation(text):
    tokens = re.split('w+', text)
    return tokens

df["word_tokens"] = df["to_lowercase"].apply(lambda x: tokenisation(x))
df.head(10)

,v1,v2,no_punctuation,to_lowercase,word_tokens
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat,go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat,"[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore , at]"
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni,"[ok lar joking , if u oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s,"[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]"
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say,[u dun say so early hor u c already then say]
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,nah i dont think he goes to usf he lives around here though,[nah i dont think he goes to usf he lives around here though]
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv,freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv,"[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent,even my brother is not like to speak with me they treat me like aids patent,"[even my brother is not like to speak , ith me they treat me like aids patent]"
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,As per your request Melle Melle Oru Minnaminunginte Nurungu Vettam has been set as your callertune for all Callers Press 9 to copy your friends Callertune,as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune,[as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune]
8,spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.,WINNER As a valued network customer you have been selected to receivea å£900 prize reward To claim call 09061701461 Claim code KL341 Valid 12 hours only,winner as a valued network customer you have been selected to receivea å£900 prize reward to claim call 09061701461 claim code kl341 valid 12 hours only,"[, inner as a valued net, ork customer you have been selected to receivea å£900 prize re, ard to claim call 09061701461 claim code kl341 valid 12 hours only]"
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,Had your mob

### Suppression des mots d'arrêt
Les mots d'arrêt sont des mots qui sont tellement communs qu'il est inutile de l'indexer et peuvent engendrer des effets négatifs sur les performances du modèle.

Pour les supprimes il faut d'abord le faire en fonction du problème que vous résolvez.


Vous pouvez les retirer en utilisant la librairie NLTK qui fournit une liste de mots d'arrêt pour les l'anglais


In [5]:
#Suppression des mots d'arret
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")

#Cette fonction permet de supprimer les stop words
def remove_stopwords(text, stopwords = stopwords):
    output = [i for i in text if i not in stopwords]
    return output

df["stop_words"] = df["word_tokens"].apply(lambda x: remove_stopwords(x))
df.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,v1,v2,no_punctuation,to_lowercase,word_tokens,stop_words
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat,go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat,"[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore , at]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]"
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni,"[ok lar joking , if u oni]","[ok lar joking , if u oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s,"[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]"
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say,[u dun say so early hor u c already then say],[u dun say so early hor u c already then say]
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,nah i dont think he goes to usf he lives around here though,[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though]
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv,freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv,"[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent,even my brother is not like to speak with me they treat me like aids patent,"[even my brother is not like to speak , ith me they treat me like aids patent]","[even my brother is not like to speak , ith me they treat me like aids patent]"
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,As per your request Melle Melle Oru Minnaminunginte Nurungu Vettam has been set as your callertune for all Callers Press 9 to copy your friends Callertune,as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune,[as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune],[as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune]
8,spam,WINNE

### La normalisation 
Comme technique de normalisation de texte il y a le **stemming** où les mots sont dérivés ou diminués à leur forme 
racine / base. ‎‎Par exemple‎‎, des mots comme « programmeur », « programmation », 
« programme » seront dérivés de « programme ».‎

Ensuite il y a la **Lemmatisation** qui tire le mot mais s’assure qu’il ne perd pas son sens.

**Lemmatization** a un dictionnaire prédéfini qui stocke le contexte des mots et vérifie le mot dans le dictionnaire 
tout en diminuant.‎

Dans notre cas nous allons utiliser la technique de **lemmatisation** Pour normaliser les données, pour cela il faudra installer wordnet.


In [6]:
#la normalisation 
from nltk.stem import WordNetLemmatizer

#installation de  wordnet
nltk.download('omw-1.4')
nltk.download('wordnet')

wordLematize = WordNetLemmatizer()

def lematizer_word(text, wordLematize=wordLematize):
    lematizer = [wordLematize.lemmatize(word) for word in text]
    return lematizer
    
df["word_lemmatize"] = df["stop_words"].apply(lambda x: lematizer_word(x))
df.head(10)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MOHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,v1,v2,no_punctuation,to_lowercase,word_tokens,stop_words,word_lemmatize
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat,go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat,"[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore , at]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]"
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni,"[ok lar joking , if u oni]","[ok lar joking , if u oni]","[ok lar joking , if u oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s,"[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]"
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say,[u dun say so early hor u c already then say],[u dun say so early hor u c already then say],[u dun say so early hor u c already then say]
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,nah i dont think he goes to usf he lives around here though,[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though]
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv,freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv,"[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent,even my brother is not like to speak with me they treat me like aids patent,"[even my brother is not like to speak , ith me they treat me like aids patent]","[even my brother is not like to speak , ith me they treat me like aids patent]","[even my brother is not like to speak , ith me they treat me like aids patent]"
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Caller

### Supprimer des liens
Pour supprimer les liens des SMS vous allez utiliser la librairie re 

In [7]:
df['no_link']=df['v2'].apply(lambda x:re.sub('http[s]?://\S+',' ', x))
df.head(10)

,v1,v2,no_punctuation,to_lowercase,word_tokens,stop_words,word_lemmatize,no_link
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat,go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat,"[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore , at]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]","Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni,"[ok lar joking , if u oni]","[ok lar joking , if u oni]","[ok lar joking , if u oni]",Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s,"[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]",Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say,[u dun say so early hor u c already then say],[u dun say so early hor u c already then say],[u dun say so early hor u c already then say],U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,nah i dont think he goes to usf he lives around here though,[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though],"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv,freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv,"[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me li

In [8]:
#Retirer les elements qui sont des caracteres, pour se faire en utilise encore la librabrie re (pas tres interressant)
df['text_no_carac']=df['to_lowercase'].apply(lambda x:re.sub(r'[\W_]+',' ',x))
df.head(10)

,v1,v2,no_punctuation,to_lowercase,word_tokens,stop_words,word_lemmatize,no_link,text_no_carac
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat,go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat,"[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore , at]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]","[go until jurong point crazy available only in bugis n great , orld la e buffet cine there got amore ]","Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni,"[ok lar joking , if u oni]","[ok lar joking , if u oni]","[ok lar joking , if u oni]",Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075over18s,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s,"[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]","[free entry in 2 a , kly comp to , in fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s]",Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say,[u dun say so early hor u c already then say],[u dun say so early hor u c already then say],[u dun say so early hor u c already then say],U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,nah i dont think he goes to usf he lives around here though,[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though],[nah i dont think he goes to usf he lives around here though],"Nah I don't think he goes to usf, he lives around here though",nah i dont think he goes to usf he lives around here though
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send å£150 to rcv,freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv,"[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std chgs to send å£150 to rcv]","[freemsg hey there darling its been 3 , eeks no, and no , ord back id like some fun you up for it still tb ok xxx std

# Vectorisation du texte
pour transformer les données textuelle en vecteurs vous pouvez utiliser l'architecture **BAG OF WORDS** avec la fonction **CountVectorizer** de
sklearn pour ensuite le passe dans fonction **tfidfTransformer** de la méthode de terme fréquent avec encore sklearn

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizerCount = CountVectorizer().fit(df["no_link"])

#Verification du nombre de carateristique dans un SMS
"il y a plus de ", len(vectorizerCount.vocabulary_), " mots"

('il y a plus de ', 8657, ' mots')

Test du transformater

In [10]:
select_sentences = df["no_link"][0]
select_sentences

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [11]:
vector = vectorizerCount.transform([select_sentences])
print(vector)

  (0, 1067)	1
  (0, 1300)	1
  (0, 1745)	1
  (0, 1747)	1
  (0, 2043)	1
  (0, 2322)	1
  (0, 3541)	1
  (0, 3585)	1
  (0, 3624)	1
  (0, 4076)	1
  (0, 4339)	1
  (0, 4465)	1
  (0, 5526)	1
  (0, 5909)	1
  (0, 7634)	1
  (0, 8018)	1
  (0, 8254)	1
  (0, 8475)	1


In [12]:
vectorData = vectorizerCount.transform(df["no_link"])
vectorData.shape

(5572, 8657)

Vous allez ensuite utilise vos donnees transformees avec CountVectorizer (bag of word) sur le transformateur de **Term Frequency (TfidfTransformer)**

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

tf = TfidfTransformer()
frequency = tf.fit_transform(vectorData)
print(frequency.shape)

(5572, 8657)


Pour la vectorisation vous pouvez utiliser un modèle prédictif appelé Word2vec pour effectuer une vectorisation sur votre DataFrame au lieu des 2 dernières méthodes.

**CE QUI MET FIN À LA VECTORISATION**

In [14]:
#Enregistrement du nouveau dataset 
df.to_csv('spam/spam_correct.csv')